## 5
### (b)

# Problem 1
## setting parameter
* There are $N$ product. $i \in \{0, 1,..., N-1\}$
* There are $M$ month. $j \in \{0, 1,..., M-1\}$, month 0, 1, 2... = March, April, March ...
* There are $K$ shipping method. $k \in \{0, 1,..., K-1\}$, shipping method 1, 2, 3 = express, air, Ocean
* Let $D_{ij}$ be the forecast demand of product $i$ of month $j$.
* Let $I_i$ be the quantity of initial inventory of product $i$.
* Let $BuyCost_i$ be the purchase cost of product $i$.
* Let $HoldCost_i$ be the holding cost of product $i$.
* Let $Transit_{ij}$ be the in-transt product $i$ which will be delivered on $j$ month.\[here $j \in\{1,2\}$\]
* Let $FixedShipCost_k$ be the fixed cost of shipping method $k$.
* Let $VarShipCost_{ik}$ be the variable cost of product $i$ of shipping method $k$.
* Let $LeadTime_k$ be the lead time of shipping method $k$.
* Let $M_{big}$ be $\sum_{j=0}^{M-1} \sum_{i=0}^{N-1} D_{ij}$

## setting decision variables
* let $x_{ijk}$ be the quantity of product $i$ that are order at month $j$ by shipping method $k$.
* let $z_{jk}$ be $1$ if at month $j$ we use shipping method $k$ else $0$

## ultility notation
* let $ExprY_{ij}$ be the expression Inventory level of product $i$ of month $j$.

$j=0:\ I_i - D_{i0} \quad \forall i$

$j=1:\ I_i - \sum_{z=0}^1 D_{iz} + x_{i00} + Transit_{i1} \quad \forall i$

$j=2:\ I_i - \sum_{z=0}^2 D_{iz} + \sum_{z=0}^1 x_{iz0} + \sum_{z=0}^0 x_{iz1} + \sum_{z=1}^2 transit_{iz} \quad \forall i$

$j \ge 3:\ I_i - \sum_{z=0}^j D_{iz} + \sum_{z=0}^{j-1} x_{iz0} + \sum_{z=0}^{j-2} x_{iz1} + \sum_{z=0}^{j-3} x_{iz2} + \sum_{z=1}^2 transit_{iz} \quad \forall i$
    

## Objective function
$$Min.\quad purchase\ Cost + Shipping\ Cost + Inventory\ Cost$$
Purchase Cost = $\sum_{i=0}^{N-1} \sum_{j=0}^{M-1} \sum_{k=0}^{K-1}( x_{ijk}\times BuyCost_i)$

Shipping Cost:<br>
* Fixed Cost = $\sum_{j=0}^{M-1} \sum_{k=0}^{K-1} (z_{jk} \times FixedShipCost_k)$

* Variable Cost = $\sum_{j=0}^{M-1} \sum_{i=0}^{N-1} \sum_{k=0}^{K-1} (x_{ijk} \times VarShipCost_{ik})$

Inventory Cost = $\sum_{i=0}^{N-1}[(\sum_{j=0}^{M-1} ExprY_{ij}) \times HoldCost_i]$

## Constrains

$ExprY_{ij} \ge 0 \quad \forall i, \forall j$

$\dfrac{\sum_{i=0}^{N-1}x_{ijk}}{M_{big}} \le z_{jk} \quad \forall j, \forall k$

$x_{ijk} \ge 0  \quad \forall i,\ \forall j,\ \forall k$

$z_{jk} \in \{0, 1\}$


In [1]:
import gurobipy as gb
from gurobipy import GRB, quicksum

In [10]:
#set data
N, M, T, S = 2, 3, 3, 3
K_i = [3, 4]
C_i = [30_000, 20_000]
P_ij = [
    [16_000, 22_000, 24_000],
    [18_000, 21_000, 20_000]
]
D_kj = [
    [8,6,7],
    [4,5,6],
    [3,2,3]
]
Q_k = [0.3, 0.6, 0.1]
y_max = sum(sum(i) for i in D_kj)

In [6]:
#create model
m = gb.Model("refinery")

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [7]:
#create variables
x = m.addVars(N, M, S, vtype=GRB.CONTINUOUS, name="x_ijk")
x

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Var *Awaiting Model Update*>}

In [8]:
y = m.addVars(N, vtype=GRB.CONTINUOUS, name="y_i")
y

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>}

In [12]:
#set objective
m.setObjective(
    quicksum([
        Q_k[k] * (
            quicksum([
                quicksum([
                    x[i,j,k] * P_ij[i][j]
                for i in range(N)
                ])
            for j in range(M)
            ])
        )
    for k in range(S)
    ]) - 
    quicksum([C_i[i] * y[i] for i in range(N)])
    ,GRB.MAXIMIZE
)

In [13]:
#add Constrains
m.addConstrs((
    quicksum([x[i,j,k] for j in range(M)]) - y[i] <= K_i[i]
    for i in range(N)
    for k in range(S)
))

m.addConstrs((
    quicksum([x[i,j,k] for i in range(N)]) <= D_kj[k][j]
    for j in range(M)
    for k in range(S)
))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>}

In [14]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 20 columns and 42 nonzeros
Model fingerprint: 0x4f57644b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+03, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 8e+00]
Presolve time: 0.01s
Presolved: 15 rows, 20 columns, 42 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2100000e+35   3.600000e+31   1.210000e+05      0s
      16    1.5610000e+05   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.02 seconds
Optimal objective  1.561000000e+05


In [17]:
for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

x_ijk[0,0,0] 0
x_ijk[0,0,1] 0
x_ijk[0,0,2] 0
x_ijk[0,1,0] 0
x_ijk[0,1,1] 0
x_ijk[0,1,2] 0
x_ijk[0,2,0] 3
x_ijk[0,2,1] 3
x_ijk[0,2,2] 3
x_ijk[1,0,0] 0
x_ijk[1,0,1] 0
x_ijk[1,0,2] 3
x_ijk[1,1,0] 5
x_ijk[1,1,1] 5
x_ijk[1,1,2] 2
x_ijk[1,2,0] 0
x_ijk[1,2,1] 0
x_ijk[1,2,2] 0
y_i[0] 0
y_i[1] 1
Obj: 156100
